# Pose Detection Task

This is a script that takes a video as input and outputs a video with pose & hand landmarks drawn to it using Google's Mediapipe library  <br>
NOTE: Providing own OAuth credentials json file is needed. Once ready, change the creds_path variable in Paths cell

Download mediapipe 

In [ ]:
!pip install mediapipe 

Download OpenCV 

In [ ]:
!pip install opencv-python

Google API & Auth

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Pose Landmarker

In [ ]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

Hand Landmarker
 

In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [ ]:
# Import necessary libraries
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

In [ ]:
# Drawing landmarks on the image 
# Provided by documentation

def draw_pose_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [ ]:
# Drawing landmarks on the image 
# Provided by documentation

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_hand_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
# Import mediapipe 
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

Paths

In [ ]:
# Tasks paths
pose_path = "pose_landmarker.task"
hand_path = "hand_landmarker.task"
creds_path = 'creds.json'  
token_path = 'token.json'  


In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
import io, os

# Allow read-only access
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# Authenticate
creds = None

if os.path.exists(token_path):
    creds = Credentials.from_authorized_user_file(token_path, SCOPES)

if not creds or not creds.valid:
    login = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
    creds = login.run_local_server(port=0)

    # Save token 
    with open(token_path, 'w') as token_file:
        token_file.write(creds.to_json())

# Connect to Drive 
connection = build('drive', 'v3', credentials=creds)
 

# List .mp4 videos 
results = connection.files().list(
    q="mimeType='video/mp4'",
    fields="files(id, name)"
).execute()

files = results.get('files', [])

if not files:
    print("No mp4 videos found.")
    exit()

print("Available videos:")
for i, f in enumerate(files):
    print(i,":",f['name'])




In [ ]:
# Choose a video
idx = int(input("\nEnter index of video: "))
file = files[idx]
selected_file = files[idx] 
file_id = selected_file['id'] 
file_name = selected_file['name']

# Download video
request = connection.files().get_media(fileId=file['id'])
with io.FileIO(file['name'], 'wb') as fh:
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()


In [ ]:

# Setting up the tasks
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

pose_options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=pose_path),
    running_mode=VisionRunningMode.VIDEO
)

hand_options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=hand_path),
    running_mode=VisionRunningMode.VIDEO)

# Capturing the video
cap = cv2.VideoCapture(file_name)

# Timestamps calculation
fps = 30
frame_time_ms = 1000.0 / fps
timestamp_ms = 0.0

#  Setting up output video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('out.mp4', fourcc, fps, (width, height))

with PoseLandmarker.create_from_options(pose_options) as pose_landmarker, HandLandmarker.create_from_options(hand_options) as hand_landmarker:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # color
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # convert to mp object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # detect poses for this frame
        pose_result = pose_landmarker.detect_for_video(mp_image, int(timestamp_ms))
        hand_result = hand_landmarker.detect_for_video(mp_image, int(timestamp_ms))

        # draw landmarks
        annotated_image = draw_pose_landmarks_on_image(mp_image.numpy_view(), pose_result)
        annotated_image = draw_hand_landmarks_on_image(annotated_image, hand_result)
        cv2.imshow("Pose Detection", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

        # RGB coloring
        annotated_bgr = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)

        out.write(annotated_bgr)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        
        timestamp_ms += frame_time_ms

cap.release()
out.release()
cv2.destroyAllWindows()


### Sample Results

![Alternative text for the image](img1.png)
![Alternative text for the image](img2.png)